In [1]:
import tensorflow as tf

import os, sys, getopt
import json
from datetime import datetime

import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K

from sklearn.utils.class_weight import compute_class_weight
from datasets import load
from densenet import densenet_model

print(tf.__version__)

2.0.0-dev20190613


In [2]:
tf.test.is_gpu_available()

True

In [3]:

# hyperparameters
# data
dataset_name = "rwth"
rotation_range = 10
width_shift_range = 0.10
height_shift_range = 0.10
horizontal_flip = True

# model
growth_rate = 128
nb_layers = [6,12]

# training
lr = 0.001
epochs = 400
max_patience = 25

# log
checkpoints = False
log_freq = 1
save_freq = 40
models_directory = 'models/'
results_directory = 'results/'
config_directory = 'config/'

general_directory = "/develop/results/"
save_directory = general_directory + "{}/dense-net/".format(dataset_name)
results = 'epoch,loss,accuracy,test_loss,test_accuracy\n'

date = datetime.now().strftime("%Y_%m_%d-%H:%M:%S")
identifier = "{}-growth-{}-densenet-{}".format(
    '-'.join([str(i) for i in nb_layers]),
    growth_rate, 
    dataset_name) + date

csv_output_map_file = save_directory + dataset_name + "_densenet.csv"
summary_file = save_directory + 'summary.csv'

In [4]:
x, y = load(dataset_name)

n_classes = len(np.unique(y))
image_shape = np.shape(x)[1:]

x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=rotation_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    horizontal_flip=horizontal_flip,
    fill_mode='constant',
    cval=0)
datagen.fit(x_train)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    fill_mode='constant',
    cval=0)
test_datagen.fit(x_train)

In [6]:
model = densenet_model(classes=n_classes, shape=image_shape, growth_rate=growth_rate, nb_layers=nb_layers)

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [11]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [12]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(images, tf.float32), training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [13]:
@tf.function
def test_step(images, labels):
    predictions = model(tf.cast(images, tf.float32), training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [14]:
# create summary writers
train_summary_writer = tf.summary.create_file_writer(save_directory + 'summaries/train/' + identifier)
test_summary_writer = tf.summary.create_file_writer(save_directory +  'summaries/test/' + identifier)

# create data generators
train_gen =  datagen.flow(x_train, y_train, batch_size=16)
test_gen = test_datagen.flow(x_test, y_test, batch_size=16, shuffle=False)

print("starting training")

min_loss = 100
min_loss_acc = 0
patience = 0

for epoch in range(epochs):
    batches = 0
    for images, labels in train_gen:
        train_step(images, labels)
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

    batches = 0
    for test_images, test_labels in test_gen:
        test_step(test_images, test_labels)
        batches += 1
        if batches >= len(x_test) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

    if (epoch % log_freq == 0):
        results += '{},{},{},{},{}\n'.format(epoch,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100)
        print ('Epoch: {}, Train Loss: {}, Train Acc:{}, Test Loss: {}, Test Acc: {}'.format(epoch,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100))

        if (test_loss.result() < min_loss):    
            if not os.path.exists(save_directory + models_directory):
                os.makedirs(save_directory + models_directory)
            # serialize weights to HDF5
            model.save_weights(save_directory + models_directory + "best{}.h5".format(identifier))
            min_loss = test_loss.result()
            min_loss_acc = test_accuracy.result()
            patience = 0
        else:
            patience += 1

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
            train_loss.reset_states()           
            train_accuracy.reset_states()           

        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            test_loss.reset_states()           
            test_accuracy.reset_states()   
            
    if checkpoints and epoch % save_freq == 0:
        if not os.path.exists(save_directory + models_directory):
            os.makedirs(save_directory + models_directory)
        # serialize weights to HDF5
        model.save_weights(save_directory + models_directory+"{}_epoch{}.h5".format(identifier,epoch))
        
    if patience >= max_patience:
        break

if not os.path.exists(save_directory + results_directory):
    os.makedirs(save_directory + results_directory)
file = open(save_directory + results_directory + 'results-'+ identifier + '.csv','w') 
file.write(results) 
file.close()

if not os.path.exists(save_directory + config_directory):
    os.makedirs(save_directory + config_directory)

config = {
    'data.dataset_name': dataset_name, 
    'data.rotation_range': rotation_range, 
    'data.width_shift_range': width_shift_range, 
    'data.height_shift_range': height_shift_range, 
    'data.horizontal_flip': horizontal_flip, 
    'model.growth_rate': growth_rate, 
    'model.nb_layers': nb_layers, 
    'train.lr': lr, 
    'train.epochs': epochs, 
    'train.max_patience': max_patience, 
}

with open(save_directory + config_directory + '.json', 'w') as json_file:
    json.dump(config, json_file)

file = open(summary_file, 'a+') 
summary = "{}, {}, dense-net, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}".format(
    date, dataset_name, save_directory + config_directory, min_loss, min_loss_acc)
file.write(summary)

file.close()

Epoch: 1, Train Loss: 3.043458938598633, Train Acc:7.462686538696289, Test Loss: 2.7706422805786133, Test Acc: 8.333333969116211
Epoch: 11, Train Loss: 2.2388598918914795, Train Acc:29.6641788482666, Test Loss: 2.798067331314087, Test Acc: 5.909090995788574
Epoch: 21, Train Loss: 1.5411089658737183, Train Acc:51.25, Test Loss: 2.986360788345337, Test Acc: 6.25
Epoch: 31, Train Loss: 1.1887325048446655, Train Acc:62.05223846435547, Test Loss: 2.3324947357177734, Test Acc: 24.772727966308594
Epoch: 41, Train Loss: 0.9962700605392456, Train Acc:68.50746154785156, Test Loss: 1.281662106513977, Test Acc: 60.871212005615234
Epoch: 51, Train Loss: 0.9263520836830139, Train Acc:71.1380615234375, Test Loss: 2.57774019241333, Test Acc: 55.30303192138672
Epoch: 61, Train Loss: 0.8348059058189392, Train Acc:74.60820770263672, Test Loss: 2.8631341457366943, Test Acc: 57.08333206176758
Epoch: 71, Train Loss: 0.77674400806427, Train Acc:77.55596923828125, Test Loss: 2.7536516189575195, Test Acc: 58.2

Epoch: 631, Train Loss: 0.10089632123708725, Train Acc:96.71641540527344, Test Loss: 0.5443257689476013, Test Acc: 86.78030395507812
Epoch: 641, Train Loss: 0.09585300832986832, Train Acc:96.75373077392578, Test Loss: 0.6266719698905945, Test Acc: 84.6212158203125
Epoch: 651, Train Loss: 0.08581169694662094, Train Acc:97.14552307128906, Test Loss: 0.49508875608444214, Test Acc: 87.46212005615234
Epoch: 661, Train Loss: 0.09028290957212448, Train Acc:96.95895385742188, Test Loss: 0.3908652365207672, Test Acc: 87.84091186523438
Epoch: 671, Train Loss: 0.08635737001895905, Train Acc:96.95895385742188, Test Loss: 0.7959036827087402, Test Acc: 81.74242401123047
Epoch: 681, Train Loss: 0.07465259730815887, Train Acc:97.5, Test Loss: 0.5715638399124146, Test Acc: 86.06060791015625
Epoch: 691, Train Loss: 0.07253492623567581, Train Acc:97.64925384521484, Test Loss: 0.4582061469554901, Test Acc: 87.42424011230469
Epoch: 701, Train Loss: 0.0960066169500351, Train Acc:96.69776153564453, Test Loss